In [1]:
import os
service_prefix = ''
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f"{service_prefix}/proxy/{{port}}"
import sys
import ee
import geemap
import numpy as np
import tqdm
import cv2
import seaborn as sns
from datetime import datetime
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from omegaconf import DictConfig, OmegaConf
import hydra
from eval.surfrad import get_emis_at, read_surfrad_file_from_url, get_surfrad_surf_temp_at
from util.ee_utils import get_landsat_capture_time, get_landsat_lst, cvt_lat_lon_to_path_row, query_geotiff

# Extracting point pixel values from an EE Image

### Test on NLCD discrete pixel values

In [2]:
lon = -95.43495
lat = 29.75765
Map = geemap.Map()
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()
# Select the land cover band.
landcover_2019 = nlcd2019.select('landcover')
# Display land cover on the map.
Map.setCenter(lon, lat, 20)
Map.addLayer(landcover_2019, None, 'Landcover')
Map

Map(center=[29.75765, -95.43495], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [3]:
point = ee.Geometry.Point(lon, lat)
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
image = dataset.filter(ee.Filter.eq('system:index', '2019')).first()
info = image.reduceRegion(ee.Reducer.first(), point, scale=30).getInfo()
pixel_value = info['landcover']
print(pixel_value)

90


It works! Note that lon, lat value of 5 decimal points (0.00001 deg, or 1m) lead to accurate result

## Test on ASTER GED

Let's try Penn State station (PSU)

In [4]:
config = OmegaConf.load('../config/surfrad.yaml')
lon = config.stations.PSU.Longitude
lat = config.stations.PSU.Latitude
print(f'lon: {lon}, lat: {lat}') 

lon: -77.93085, lat: 40.72012


In [5]:
vis_params = {
    'min': 0.9,   # Set the minimum emissivity value you expect to see
    'max': 1.0,   # Set the maximum emissivity value you expect to see
    'palette': ['blue', 'green', 'yellow', 'red']  # Define a color palette from low to high emissivity
}

Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(lon, lat, 20)
image = ee.Image('NASA/ASTER_GED/AG100_003').select('emissivity_band10').multiply(0.001)
Map.addLayer(image, vis_params, 'emis')
Map

Map(center=[40.72012, -77.93085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [6]:
point = ee.Geometry.Point(lon, lat)
image = ee.Image('NASA/ASTER_GED/AG100_003').multiply(0.001)
info = image.reduceRegion(ee.Reducer.first(), point, scale=1).getInfo()
emis_10 = info['emissivity_band10']
emis_11 = info['emissivity_band11']
emis_12 = info['emissivity_band12']
emis_13 = info['emissivity_band13']
emis_14 = info['emissivity_band14']
print(emis_10, emis_11, emis_12, emis_13, emis_14)

0.967 0.964 0.961 0.974 0.974


In [7]:
broadband_emis = get_emis_at(lon, lat)
print('broadband_emis = ', broadband_emis)

broadband_emis =  0.9716729999999999


## Test on user-uploaded image

In [67]:
vis = {
  'min': 270,
  'max': 330,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}


Map = geemap.Map()
Map.add_basemap('HYBRID')
img_path = '/home/yuhaoliu/Code/UrbanSurfTemp/data/Austin/lst/LC08_ST_B10_20170303.tif'
output_path = '/home/yuhaoliu/Code/UrbanSurfTemp/data/Austin/output_referenced_f75/lst/20170303.tif'
image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_027039_20170303')
Map.add_raster(img_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='downloaded_lst')
Map.add_raster(output_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='output_lst')
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [69]:
meta_ = Map.user_rois.getInfo()
coord = meta_['features'][0]['geometry']['coordinates']
lon, lat = coord[0], coord[1]
print(f'lon = {lon}, lat = {lat}')

landsat_lst = get_landsat_lst(lon, lat, image=image)
capture_time = get_landsat_capture_time(image=image)
print(f'LANDSAT LST = {landsat_lst} K')
geotiff_lst = query_geotiff(lon, lat, img_path)
print(f'GeoTIFF LST = {geotiff_lst}')
output_lst = query_geotiff(lon, lat, output_path)
print(f'Output GeoTIFF LST = {output_lst}')

lon = -97.720814, lat = 30.375057
LANDSAT LST = 299.60137922 K
GeoTIFF LST = 299.53301882
Output GeoTIFF LST = 299.53301882


There is a pixel shift of about half a pixel (15m) between original LANDSAT image and the cropped & downloaded landsat image.</br>
There is no shfit between downloaded image and island output

#### What about SURFRAD sites?

In [ ]:
Map.set_center(lon, lat, 17)

In [2]:
vis = {
  'min': 280,
  'max': 340,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}
config = OmegaConf.load('../config/surfrad.yaml')
station_id = 'BND'
date_ = '20170611'
lon = config['stations'][station_id]['Longitude']
lat = config['stations'][station_id]['Latitude']
scene_id = config['stations'][station_id]['scene_id']
Map = geemap.Map()
Map.add_basemap('HYBRID')
img_path = f'/home/yuhaoliu/Code/UrbanSurfTemp/data/{station_id}/lst/LC08_ST_B10_{date_}.tif'
# output_path = '/home/yuhaoliu/Code/UrbanSurfTemp/data/Austin/output_referenced_f75/lst/20170303.tif'
image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_{scene_id}_{date_}')
Map.add_raster(img_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='downloaded_lst')
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [82]:
meta_ = Map.user_rois.getInfo()
coord = meta_['features'][0]['geometry']['coordinates']
lon, lat = coord[0], coord[1]
print(f'lon = {lon}, lat = {lat}')

landsat_lst = get_landsat_lst(lon, lat, image=image)
capture_time = get_landsat_capture_time(image=image)
print(f'LANDSAT LST = {landsat_lst} K')
geotiff_lst = query_geotiff(lon, lat, img_path)
print(f'GeoTIFF LST = {geotiff_lst}')

lon = -88.373294, lat = 40.05213
LANDSAT LST = 314.90385476 K
GeoTIFF LST = 314.16214442


In [53]:
#nah
image.projection().getInfo()['transform']

[30, 0, 495885, 0, -30, 3468915]

In [57]:
geemap.ee_export_image(image.select('ST_B10'), 
                       filename='/home/yuhaoliu/Code/UrbanSurfTemp/tmp/test.tif', 
                       # scale=30, 
                       region="[[[-97.879991, 30.157262], [-97.562735, 30.157262], [-97.562735, 30.580256], [-97.879991, 30.580256], [-97.879991, 30.157262]]]",
                       crs=image.projection().getInfo()['crs'],
                       crs_transform = image.projection().getInfo()['transform'],
                       file_per_band=False)

Generating URL ...
An error occurred while downloading.
Total request size (172158180 bytes) must be less than or equal to 50331648 bytes.


In [64]:
# this works, without region clip
geemap.ee_export_image_to_drive(image.select('ST_B10'),
                                description='landsat', folder='export2',
                       # filename='/home/yuhaoliu/Code/UrbanSurfTemp/tmp/test.tif', 
                       # scale=30, 
                       # region="[[[-97.879991, 30.157262], [-97.562735, 30.157262], [-97.562735, 30.580256], [-97.879991, 30.580256], [-97.879991, 30.157262]]]",
                       crs=image.projection().getInfo()['crs'],
                       crs_transform = image.projection().getInfo()['transform'])

In [65]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
image = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_027039_20170303')
tmp_raster_path = '/home/yuhaoliu/Code/UrbanSurfTemp/tmp/entire_img.tif'
tmp_raster_path2 = '/home/yuhaoliu/Code/UrbanSurfTemp/tmp/test_mis_aligned.tif'
Map.addLayer(image.select('ST_B10').multiply(0.00341802).add(149), vis, 'landsat_lst')
Map.add_raster(tmp_raster_path, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='tmp_lst')
Map.add_raster(tmp_raster_path2, palette=vis['palette'], vmax=vis['max'], vmin=vis['min'], layer_name='tmp_lst_mis')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# SURFRAD Validation

## What is the ASTER GEDv3 Emissivity for all SURFRAD stations?

In [11]:
for station in config.stations:
    lon = config['stations'][station]['Longitude']
    lat = config['stations'][station]['Latitude']
    emis = get_emis_at(lon, lat)
    print(f'{station} station broadband emissivity = {emis:.4f}')

BND station broadband emissivity = 0.9751
TBL station broadband emissivity = 0.9711
DRA station broadband emissivity = 0.9698
FPK station broadband emissivity = 0.9791
GWN station broadband emissivity = 0.9690
PSU station broadband emissivity = 0.9717
SXF station broadband emissivity = 0.9727
SGP station broadband emissivity = 0.9743


## What does the land cover looklike at SURFRAD stations?

In [12]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Select the land cover band.
landcover_2019 = nlcd2019.select('landcover')

# Display land cover on the map.
lon = config.stations.PSU.Longitude
lat = config.stations.PSU.Latitude
Map.setCenter(lon, lat, 15)
Map.addLayer(landcover_2019, None, 'Landcover')
Map

Map(center=[40.72012, -77.93085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

Not diverse at all...

## Compare SURFRAD with LANDSAT on a clear day

We use (lon, lat) -> (path, row) converter here: https://landsat.usgs.gov/landsat_acq#convertPathRow </br>
For PSU, we have Path: 16, Row: 32

In [3]:
vis = {
  'min': 250,
  'max': 300,
  'palette' : sns.color_palette('inferno', 20).as_hex(),
}


Map = geemap.Map()
Map.setCenter(lon, lat, 15)
Map.add_basemap('HYBRID')
date_ = '20191123'
landsat = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_016032_{date_}').select('ST_B10').multiply(0.00341802).add(149)
Map.addLayer(landsat, vis)
Map

Map(center=[40.72012, -77.93085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

### dev

In [14]:
point = ee.Geometry.Point(lon, lat)
info = landsat.reduceRegion(ee.Reducer.first(), point, scale=30).getInfo()
pixel_value = info['ST_B10']
print('LST in kelvin via LANDSAT 8: ', pixel_value)

LST in kelvin via LANDSAT 8:  278.35838492


In [10]:
image  = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_016032_{date_}')
# Get the capture time
capture_time = image.get('system:time_start')
# Retrieve the value from the server
capture_time_info = capture_time.getInfo()
# Convert the timestamp to minutes, round it, and then convert back to milliseconds
rounded_time = ee.Number(capture_time_info).divide(1000 * 60).round().multiply(1000 * 60)
# Convert the rounded timestamp to a readable date
rounded_date = ee.Date(rounded_time).format('YYYY-MM-dd HH:mm').getInfo()
dt = datetime.strptime(rounded_date, '%Y-%m-%d %H:%M')
print(dt)

NameError: name 'date_' is not defined

In [76]:
# getting jday
dt.timetuple().tm_yday

327

In [90]:
station_id = 'psu'
year_ = dt.year
year2 = str(dt.year)[-2:]
jday = dt.timetuple().tm_yday
url = f'https://gml.noaa.gov/aftp/data/radiation/surfrad/{station_id}/{dt.year}/{station_id}{year2}{jday}.dat'
print(url)
df = read_surfrad_file_from_url(config, url)
df

https://gml.noaa.gov/aftp/data/radiation/surfrad/psu/2019/psu19327.dat


,year,jday,month,day,hour,min,dt,zen,dw_solar,qc_dwsolar,...,windspd,qc_windspd,winddir,qc_winddir,pressure,qc_pressure,station_name,latitude,longitude,elevation
0,2019,327,11,23,0,0,0.000,114.48,-1.7,0,...,3.8,0,304.7,0,973.2,0,Penn State,40.72,376.0,1.0
1,2019,327,11,23,0,1,0.017,114.67,-1.7,0,...,4.8,0,297.3,0,973.2,0,Penn State,40.72,376.0,1.0
2,2019,327,11,23,0,2,0.033,114.86,-1.7,0,...,4.7,0,284.8,0,973.2,0,Penn State,40.72,376.0,1.0
3,2019,327,11,23,0,3,0.050,115.05,-1.7,0,...,2.9,0,294.3,0,973.3,0,Penn State,40.72,376.0,1.0
4,2019,327,11,23,0,4,0.067,115.24,-1.7,0,...,3.6,0,311.1,0,973.3,0,Penn State,40.72,376.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2019,327,11,23,23,55,23.917,113.62,-1.3,0,...,1.8,0,39.7,0,962.8,0,Penn State,40.72,376.0,1.0
1436,2019,327,11,23,23,56,23.933,113.80,-1.3,0,...,1.9,0,97.9,0,962.8,0,Penn State,40.72,376.0,1.0
1437,2019,327,11,23,23,57,23.950,113.99,-1.3,0,...,1.4,0,129.6,0,962.8,0,Penn State,40.72,376.0,1.0
1438,2019,327,11,23,23,58,23.967,114.18,-1.3,0,...,0.6,0,150.1,0,962.7,0,Penn State,40.72,376.0,1.0


In [106]:
row = df[df['hour'] == dt.hour]
row = row[row['min'] == dt.minute]
uw_ir = row['uw_ir'].iloc[0]
dw_ir = row['dw_ir'].iloc[0]
air_temp = row['temp'].iloc[0]
print('upwelling thermal infrared (Watts m^-2): ', uw_ir)
print('downwelling thermal infrared (Watts m^-2): ', dw_ir)
print('10-m air temperature (Celcius): ', air_temp)

upwelling thermal infrared (Watts m^-2):  347.9
downwelling thermal infrared (Watts m^-2):  228.8
10-m air temperature (Celcius):  2.4


In [117]:
def calc_lst(emis, sigma, uw_ir, dw_ir):
    lst = ((1 / (emis * sigma)) * (uw_ir - (1-emis) * dw_ir))**0.25
    return lst

sigma = 5.670374419e-8 # Stefan–Boltzmann constant
lst = calc_lst(broadband_emis, sigma, uw_ir, dw_ir)
print('SURFRAD LST: ', lst)
print('LST in kelvin via LANDSAT 8: ', pixel_value)
print('error = ', lst - pixel_value)

SURFRAD LST:  280.56848324625804
LST in kelvin via LANDSAT 8:  278.35838492
error =  2.2100983262580485


### Results

In [2]:
ee.Initialize()
config = OmegaConf.load('../config/surfrad.yaml')
lon = config.stations.PSU.Longitude
lat = config.stations.PSU.Latitude
date_ = '20191123'
image  = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_016032_{date_}')
landsat_lst = get_landsat_lst(lon, lat, image=image)
capture_time = get_landsat_capture_time(image=image)
print(f'LANDSAT LST = {landsat_lst} K')
surfrad_lst = get_surfrad_surf_temp_at('PSU', capture_time)
print(f'SURFRAD LST = {surfrad_lst} K')

LANDSAT LST = 278.35838492 K
SURFRAD LST = 280.5678033598088 K


In [ ]:
# a different day
Map = geemap.Map()
Map.setCenter(lon, lat, 17)
Map.add_basemap('HYBRID')
date_ = '20190515'
landsat = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_016032_{date_}').select('ST_B10').multiply(0.00341802).add(149)
Map.addLayer(landsat, vis)
Map

In [8]:
# a different day
image  = ee.Image(f'LANDSAT/LC08/C02/T1_L2/LC08_016032_{date_}')
landsat_lst = get_landsat_lst(lon, lat, image=image)
capture_time = get_landsat_capture_time(image=image)
print(f'LANDSAT LST = {landsat_lst} K')
surfrad_lst = get_surfrad_surf_temp_at('PSU', capture_time)
print(f'SURFRAD LST = {surfrad_lst} K')

LANDSAT LST = 296.10474476 K
SURFRAD LST = 294.530739581366 K


## Path Row numbers for the surfrad stations

In [3]:
config = OmegaConf.load('../config/surfrad.yaml')
for station in config.stations:
    lon, lat = config['stations'][station]['Longitude'], config['stations'][station]['Latitude']
    print(f'{station}: \nlon = {lon}, lat = {lat}')
    try:
        r = cvt_lat_lon_to_path_row(lat, lon)
        path, row = r['path'], r['row']
        print(f'path = {path}, row = {row}')
    except ValueError as e:
        pass
    finally:
        print('')

BND: 
lon = -88.37309, lat = 40.05192
path = 23, row = 32

TBL: 
lon = -105.2368, lat = 40.12498
path = 34, row = 32

DRA: 
lon = -116.01947, lat = 36.62373
path = 40, row = 35

FPK: 
lon = -105.1017, lat = 48.30783
path = 36, row = 26

GWN: 
lon = -89.8729, lat = 34.2547
path = 23, row = 36

PSU: 
lon = -77.93085, lat = 40.72012
path = 16, row = 32

SXF: 
lon = -96.62328, lat = 43.73403
path = 29, row = 30

SGP: 
lon = -97.48525, lat = 36.60406
path = 28, row = 35

